<a href="https://colab.research.google.com/github/OasisLead/Hello-world/blob/main/Datacamp_project_Lacroix_Louis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of the project is to train machine learning models on a EEG dataset(SVM and KNN, could have used a neural network but it is not very useful in this task since the data is pretty small in size, and also since I used Fourier transform to extract the relevant EEG frequencies, I don't use "time series" techniques such as LSTM for example).

I first pre-process the EEG data set by choosing relevant channels, removing noise, choosing relevant data and doing a fourier transform to obtain the intensity of the different frequencies (which is what we are interested in when we study a general EEG state, rather than a event related potential).
The data set consists of EEG data from different experiments, in which multiple participants were recorded for 40 minutes during tasks that demanded various levels of attention.
The EEG data has 3 labels : focused (1st 10 minutes), unfocused (10->20 minutes), drowsed (20->40). We will use these labels for classification.

Then after this pre-processing, I did multiple PCA pipelines with 4 sizes of basis, and trained SVM and KNN on them. I actually needed to do PCA if I wanted to algorithm to converge in less than 10 minutes, since the data is pretty multidimensional, and since I wanted a good accuracy.

The results show a great accuracy, which is kind of surprising, and I don't have explanations for it. Surpisingly the KNN has a better accuracy than the SVM, while converging faster, this I can't really explain. Moreover, the SVM has a better accuracy with increasing PCA basis.

I could have done cross-validation or more detailled analysis, to see if my models are generalizable. The point of this was really to make the pre-processing with fourier transform and make the whole pipeline, not to find the best, fastest, and most generalizable model.
One idea for further development is that if you do this analysis in real time in a patient, you want the analysis to be pretty fast, so to improve maybe on model size (smaller model for faster computations) and model complexity, and also the analysis doesn't need to take only 10 seconds of sample but can take like 10 minutes of sample, so gather more data before choosing the actual category by choosing the most frequent label in the last 10 minutes for example.





# Librairies and data importation

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.signal import stft
import os
import sklearn
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import sys
import pickle
#!pip install tensorflow
import tensorflow as tf
#!pip install keras
from tensorflow import keras
from tensorflow.keras import layers

# Connexion to drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading database files

In [ ]:
#create a list of files name
data_dir = []

# set the dataset's path
data_path = './drive/My Drive/Colab Notebooks/EEG Data/'

number_of_recordings = 34

#set the data dir as the name of all files in the directory
data_dir = ["eeg_record" + str(k) for k in range(1,number_of_recordings +1)]




#load data from each files in d[name]
d = {}
for name in data_dir:
    d[name] = loadmat(data_path + name + '.mat')

# Extraction of signal in recordings and removing bad channels

Some variable instantiation

In [ ]:
# sampling frequency is 128 Hz
fs = 128
# recording last for 40 min
sample_nbr = 128 * 40 * 60
#sample_nbr is the number of sample for a given signal

#1 recording has multiple signal

print('Total number of sample for 1 signal is' , sample_nbr)

Total number of sample for 1 signal is 307200




Separation of useful signal from recordings:

I kept only the relevant channels (Neuroscience criterion from the paper of the data set)

Creation of the dictionnary that will store data

In [ ]:
#available signals in the loaded database for 1 recording
signals = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
#signals to use in the loaded database for 1 recording ()
useful_signals = ['F7','F3','P7','O1','O2','P8','AF4']

#extract index number for useful signals
use_signal_inds = []
for c in useful_signals:
    if c in signals:
        use_signal_inds.append(signals.index(c))

# create a dictionnary for the useful files data
eeg = dict()

for name in data_dir:
    eeg[name] = dict()
    eeg[name] = d[name]['o']['data'][0][0][0:sample_nbr, 3:17]
    eeg[name] = eeg[name][:,use_signal_inds]

print('shape of a recording is now ' ,eeg['eeg_record1'].shape )

shape of a recording is now  (307200, 7)


Checking data quality and removing incomplete data

In [ ]:
# Removing incomplete channels
for name in data_dir:
  data_time = len(eeg[name])/128/60
  print('Name : ',name,', Data time:',data_time,'minutes ')

##Delete the bad channels from data_dir
for record in ["eeg_record28", "eeg_record6", "eeg_record16"]:
    if record in data_dir:
        data_dir.remove(record)

print("-------------------------------------------------------------------------------")
for name in data_dir:
  data_time = len(eeg[name])/128/60
  print('Name : ',name,', Data time:',data_time,'minutes ')

Name :  eeg_record1 , Data time: 40.0 minutes 
Name :  eeg_record2 , Data time: 40.0 minutes 
Name :  eeg_record3 , Data time: 40.0 minutes 
Name :  eeg_record4 , Data time: 40.0 minutes 
Name :  eeg_record5 , Data time: 40.0 minutes 
Name :  eeg_record6 , Data time: 37.59791666666667 minutes 
Name :  eeg_record7 , Data time: 40.0 minutes 
Name :  eeg_record8 , Data time: 40.0 minutes 
Name :  eeg_record9 , Data time: 40.0 minutes 
Name :  eeg_record10 , Data time: 40.0 minutes 
Name :  eeg_record11 , Data time: 40.0 minutes 
Name :  eeg_record12 , Data time: 40.0 minutes 
Name :  eeg_record13 , Data time: 40.0 minutes 
Name :  eeg_record14 , Data time: 40.0 minutes 
Name :  eeg_record15 , Data time: 40.0 minutes 
Name :  eeg_record16 , Data time: 30.5171875 minutes 
Name :  eeg_record17 , Data time: 40.0 minutes 
Name :  eeg_record18 , Data time: 40.0 minutes 
Name :  eeg_record19 , Data time: 40.0 minutes 
Name :  eeg_record20 , Data time: 40.0 minutes 
Name :  eeg_record21 , Data ti

# Defining feature extraction methods


Define the Smoothing function to remove noise

In [ ]:
#To clean the EEG data

def moving_average_smooth(interval, windowsize):
    window = np.ones(int(windowsize)) / float(windowsize)
    re = np.convolve(interval, window, 'same') # convolve(a,v): return discrete, linear convolution of a and v.
    return re

In [ ]:
#Spectrogram function = the square of the STFT amplitudes
func = lambda x:(np.abs(x))**2

Creating the feature_extraction function to do:
- Short time Fourier transform over a time window to get the power of the different frequencies in a short term range and store them as the relevant values
- Restrict frequency range
- Smooth the data and remove noise
- Label the data and store the labels

In [ ]:

def feature_extraction(input_data,window_size):
    feature_list = []
    current_time = 0
    label = list()

    ## FEATURE
    #for each signal in the recording
    for i in range(7):

        #calculate the short time fourrier transformation
        eeg_feq = stft(input_data[:,i],128,'blackman')

        sample_freq = eeg_feq[0]  # Array of sample frequencies.
        segment_time = eeg_feq[1] # Array of segment times.
        stft_result = eeg_feq[2]  # STFT of `x`

        #restrict the frequency range, specific to EEG signal to remove noise
        restricted_stft_result = stft_result[1:37,:] # select only frequency between 0.5 and 18Hz with a 0.5 step

        # for each frequency, smoothe the  amplitude signal using a 15 s-running average.
        for j in range(36):
            current = np.apply_along_axis(func, axis=0, arr = restricted_stft_result[j,:] ) #apply square to STFT applitude
            feature = moving_average_smooth(current,window_size*128) # apply average smooth to squared stft amplitude
            feature_list.append(feature) # add the result to the feature_list

    ## LABEL
    data_time = len(input_data)/128/60
    # 3 labels : focused -> 0 | unfocused -> 1 | drowsed -> 2
    for m in range(len(segment_time)):
        current_time += data_time/len(segment_time)  # 40 minutes / 2401
        if current_time < 10:
            label.append(0) #focused
        elif 10 < current_time < 20:
            label.append(1) #unfocused
        else:
            label.append(2) #drowsed


    feature_list = np.array(feature_list)
    label = np.array(label)
    #print(feature_list.shape); # (252, 2401) 252 = 7 * 36

    return feature_list, label

Scaling / Averaging pre-processing

In [ ]:
def minmaxscaler_dataframe_train(feature_list):
    scaler_list = list()
    scaled_feature_list = list()

    #for data received from an stft
    for i in range(len(feature_list)): #operation done 2401 times
        scaler = MinMaxScaler()
        x = np.array(feature_list[i]).reshape(-1,1) #set the 252 sample list as a column vector
        x = scaler.fit_transform(x)                 #x has 252 lines and 1 column, contains values form 0 to 1
        scaled_feature_list.append(x.reshape(-1))   #add to the list x as a single line
        scaler_list.append(scaler)

    scaled_feature_list = np.array(scaled_feature_list)
    scaler_list = np.array(scaler_list)

    return scaled_feature_list, scaler_list

Final pre-processing function pipeline using previously made functions

In [ ]:
def data_extraction_scaling(dirname,window_size) :

  #create unvalid 252 vector
  feature_list = np.empty([252, 1])
  label_list = []

  #define feature_list as a array
  feature_list = np.array(feature_list)

  for name in data_dir:
    #apply feature_extraction
    feature, label = feature_extraction(eeg[name],window_size)
    label_list = np.append(label_list, label)

    #apply minMaxScaler
    scaled_feature, scaler = minmaxscaler_dataframe_train(feature)

    #add extracted feature to the list
    feature_list = np.append(feature_list, scaled_feature, axis = 1)
    #reshape feature list to have vectors of 252 elements
    feature_list = feature_list.reshape(252,-1)

  #remove the first unvalid 252 vector
  feature_list = feature_list[ :, 1: ];

  #define label_list as a array
  label_list = np.array(label_list).reshape(-1,1 )

  return feature_list.transpose(), label_list

# Feature extraction tests

Test of the shape of our data

In [ ]:
test1, test2 = feature_extraction(eeg['eeg_record1'],15)
print('For a given recording the feature format is' ,test1.shape)
print('For the label associated we have',test2.shape)

For a given recording the feature format is (252, 2401)
For the label associated we have (2401,)


Test after minmaxscaler

In [ ]:
test3, test4 = minmaxscaler_dataframe_train(test1)
print('For a given signal the scaled feature format is ', test3.shape)
print('The scaler list has the format ', test4.shape)

For a given signal the scaled feature format is  (252, 2401)
The scaler list has the format  (252,)


# Final data extraction and main pre-processing

In [ ]:
feature_list, label_list = data_extraction_scaling(data_dir, 15)

print('Feature_list has a shape of ', feature_list.shape)
print('Label_list has a shape of', label_list.shape)

print('\n\rOne element of the feature_list has a len of ', len(feature_list[0]))
print('One element of the feature_list has a shape of ', feature_list[0].shape)

print('\n\rOne element of the label_list has a len of', len(label_list[0]))
print('One element of the label_list has a shape of ', label_list[0].shape)

Feature_list has a shape of  (74431, 252)
Label_list has a shape of (74431, 1)

One element of the feature_list has a len of  252
One element of the feature_list has a shape of  (252,)

One element of the label_list has a len of 1
One element of the label_list has a shape of  (1,)


# Neural network data preparation

### Creating Training and Test set, with different PCA


Creating 4 PCA models of respectively 4, 10, 20 and 100 dimensions

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

# feature_list.shape -> (74431, 252)
# label_list.shape -> (74431, 1)

pca = PCA(n_components=4)
features_reduced = pca.fit_transform(feature_list)

pca1 = PCA(n_components=10)
features_reduced1 = pca1.fit_transform(feature_list)

pca2 = PCA(n_components=20)
features_reduced2 = pca2.fit_transform(feature_list)

pca3 = PCA(n_components=100)
features_reduced3 = pca3.fit_transform(feature_list)

Using 4 PCA models to make 4 training-test data sets

In [ ]:
X_train_PCA, X_test_PCA, Y_train_PCA, Y_test_PCA = train_test_split(np.array(features_reduced), np.array(label_list), test_size=0.2, random_state=42)
print('Input feature shapes:',X_train_PCA.shape, X_test_PCA.shape)
print('Label output shape:',Y_train_PCA.shape, Y_test_PCA.shape)

X_train_PCA1, X_test_PCA1, Y_train_PCA1, Y_test_PCA1 = train_test_split(np.array(features_reduced1), np.array(label_list), test_size=0.2, random_state=42)
print('Input feature shapes:',X_train_PCA1.shape, X_test_PCA1.shape)
print('Label output shape:',Y_train_PCA1.shape, Y_test_PCA1.shape)

X_train_PCA2, X_test_PCA2, Y_train_PCA2, Y_test_PCA2 = train_test_split(np.array(features_reduced2), np.array(label_list), test_size=0.2, random_state=42)
print('Input feature shapes:',X_train_PCA2.shape, X_test_PCA2.shape)
print('Label output shape:',Y_train_PCA2.shape, Y_test_PCA2.shape)

X_train_PCA3, X_test_PCA3, Y_train_PCA3, Y_test_PCA3 = train_test_split(np.array(features_reduced3), np.array(label_list), test_size=0.2, random_state=42)
print('Input feature shapes:',X_train_PCA3.shape, X_test_PCA3.shape)
print('Label output shape:',Y_train_PCA3.shape, Y_test_PCA3.shape)

Input feature shapes: (59544, 4) (14887, 4)
Label output shape: (59544, 1) (14887, 1)
Input feature shapes: (59544, 10) (14887, 10)
Label output shape: (59544, 1) (14887, 1)
Input feature shapes: (59544, 20) (14887, 20)
Label output shape: (59544, 1) (14887, 1)
Input feature shapes: (59544, 100) (14887, 100)
Label output shape: (59544, 1) (14887, 1)


# Neural network training and testing

### SVM

1) Creating different models for each PCA pipeline

2) Training the Models

3) Getting the prediction of the test set

4) Printing accuracy on test set


In [ ]:
classifier9 = LinearSVC(C=1, random_state=0)

classifier0 = OneVsRestClassifier(classifier9,n_jobs=-1)
classifier1 = OneVsRestClassifier(classifier9,n_jobs=-1)
classifier2 = OneVsRestClassifier(classifier9,n_jobs=-1)
classifier3 = OneVsRestClassifier(classifier9,n_jobs=-1)

# classifier = svm.SVC(kernel='linear') # Linear Kernel
classifier0.fit(X_train_PCA, Y_train_PCA)
classifier1.fit(X_train_PCA1, Y_train_PCA1)
classifier2.fit(X_train_PCA2, Y_train_PCA2)
classifier3.fit(X_train_PCA3, Y_train_PCA3)
#Prediction for the Test set
Y_pred_svm_PCA0 = classifier0.predict(X_test_PCA)
Y_pred_svm_PCA1 = classifier1.predict(X_test_PCA1)
Y_pred_svm_PCA2 = classifier2.predict(X_test_PCA2)
Y_pred_svm_PCA3 = classifier3.predict(X_test_PCA3)

print('Accuracy score PCA 4: ',accuracy_score(Y_pred_svm_PCA0, Y_test_PCA))
print('Accuracy score PCA 10: ',accuracy_score(Y_pred_svm_PCA1, Y_test_PCA1))
print('Accuracy score PCA 20: ',accuracy_score(Y_pred_svm_PCA2, Y_test_PCA2))
print('Accuracy score PCA 100: ',accuracy_score(Y_pred_svm_PCA3, Y_test_PCA3))

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Accuracy score PCA 4:  0.7481695438973601
Accuracy score PCA 4:  0.8109760193457379
Accuracy score PCA 4:  0.8559817290253241
Accuracy score PCA 4:  0.9717874655739908


I focus only on PCA, since if I don't it takes way too many time to fit the classifier

In [ ]:
# Fit du  Training set
# classifier1 = SVC(kernel = 'linear', random_state = 0)
#classifier1 = LinearSVC(C=1, random_state=0)
#classifier = OneVsRestClassifier(classifier1,n_jobs=-1)

# classifier = svm.SVC(kernel='linear') # Linear Kernel
#classifier.fit(X_train, Y_train)
#Prediction for the Test set
#Y_pred_svm = classifier.predict(X_test)

#print('Accuracy score: ',accuracy_score(Y_pred_svm, Y_test))#  classifier.score(X_test,Y_test)
# from sklearn.metrics import classification_report
# print(classification_report(Y_test, Y_pred_svm))

### KNN

1) Creating different models for each PCA pipeline

2) Training the Models

3) Printing accuracy on test set

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

estimator = KNeighborsClassifier()
estimator1 = KNeighborsClassifier()
estimator2 = KNeighborsClassifier()
estimator3 = KNeighborsClassifier()

estimator.fit(X_train_PCA, Y_train_PCA)
estimator1.fit(X_train_PCA1, Y_train_PCA1)
estimator2.fit(X_train_PCA2, Y_train_PCA2)
estimator3.fit(X_train_PCA3, Y_train_PCA3)

# print(y_valid)
print('Accuracy score: ', estimator.score(X_test_PCA, Y_test_PCA))
print('Accuracy score: ', estimator1.score(X_test_PCA1, Y_test_PCA1))
print('Accuracy score: ', estimator2.score(X_test_PCA2, Y_test_PCA2))
print('Accuracy score: ', estimator3.score(X_test_PCA3, Y_test_PCA3))

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for 

Accuracy score:  0.999126754886814
Accuracy score:  0.9989252367837711
Accuracy score:  0.9990595821857997
Accuracy score:  0.999126754886814
